# Backtesting a Trading Strategy Part 2

## Or How to Retrieve S&P Constituents Historical Data Using Python Package pandas-datareader

## Introduction
Backtesting is a tool we can use to measure the performance of a trading strategy using historical data. The backtesting process can be broken down in three parts: first, we need to decide the universe of securities where we will invest in (e.g. equity or fixed income? US or emerging markets?); second, we need to retrieve historical data for the universe of securities in the previous step; and last part is where we implement our trading strategy using the data collected. 

Recall that in the [previous article](), we tackled the initial step in the backtesting process, that is, we determined the universe of stocks: S&P 500; S&P MidCap 400; and S&P SmallCap 600. In this article, we will discuss the second step in the backtesting procedure, that is, we will retrieve historical data for each constituent on all the indices. To achieve this feat, we will use Python package [pandas-datareader]() to collect historical data from the [Investors Exchange (IEX)]() which spans the last 5 years.

## Retrieve Historical Data

You can find the code below on https://github.com/DinodC/backtesting-mean-reverting-strategy.

Import packages

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
from pandas import Series, DataFrame

In [4]:
import pickle

In [5]:
import pandas_datareader.data as web

In [127]:
%matplotlib inline

### S&P Constituents Tickers
In this section, we load the lists pickled from the last article.

Set an id for each index

In [6]:
id = ['sp500', 'sp400', 'sp600']

Create a dictionary to map each id to a tickers file

In [7]:
input_file = {'sp500': 'sp500_barchart.pickle',
              'sp400': 'sp400_barchart.pickle', 
              'sp600': 'sp600_barchart.pickle'} 

Define a dictionary to map each id to a tickers list

In [8]:
sp500_tickers = []
sp400_tickers = []
sp600_tickers = []
sp_tickers = {'sp500': sp500_tickers,
              'sp400': sp400_tickers,
              'sp600': sp600_tickers}

Fill the tickers lists

In [9]:
for i in input_file:
    with open(input_file[i], 'rb') as f:
        
        # Update tickers list        
        sp_tickers[i] = pickle.load(f)

        # Sort tickers list
        sp_tickers[i].sort()
        
    f.close()

### S&P Constituents Historical Data 

Define dictionary of historical data

In [10]:
sp500_data = pd.DataFrame()
sp400_data = pd.DataFrame()
sp600_data = pd.DataFrame()
sp_data = {'sp500': sp500_data,
           'sp400': sp400_data,
           'sp600': sp600_data}

Set the start and date of the historical data

In [11]:
start_date = '2014-01-01'
end_date = '2020-01-01'

Set the source [Investors Exchange (IEX)](https://iextrading.com) to be used

In [12]:
source = 'iex'

Create a dictionary to map each id to an output file

In [13]:
output_file = {'sp500': 'sp500_data.pickle',
               'sp400': 'sp400_data.pickle',
               'sp600': 'sp600_data.pickle'}

Retrieve historical data for each constituent of each S&P index

In [112]:
for i in output_file:
    
    # Retrieve historical data 
    # Note that we set number of tickers to < 100 because DataReader gives error when number of tickers > 100
    data1 = web.DataReader(sp_tickers[i][:98], source, start_date, end_date)
    data2 = web.DataReader(sp_tickers[i][98:198], source, start_date, end_date)
    data3 = web.DataReader(sp_tickers[i][198:298], source, start_date, end_date)
    data4 = web.DataReader(sp_tickers[i][298:398], source, start_date, end_date)
    data5 = web.DataReader(sp_tickers[i][398:498], source, start_date, end_date)
    if i == 'sp400':
        # Concatenate historical data
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5], axis=1, sort=True)
    if i == 'sp500':
        data6 = web.DataReader(sp_tickers[i][498:], source, start_date, end_date)
        # Concatenate historical data        
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6], axis=1, sort=True)
    elif i == 'sp600':
        data6 = web.DataReader(sp_tickers[i][498:598], source, start_date, end_date)
        data7 = web.DataReader(sp_tickers[i][598:], source, start_date, end_date)
        # Concatenate historical data
        sp_data[i] = pd.concat([data1, data2, data3, data4, data5, data6, data7], axis=1, sort=True)        
    else:
        pass
            
    # Convert index to datetime
    sp_data[i].index = pd.to_datetime(sp_data[i].index)
    
    # Save historical data to file
    with open(output_file[i], 'wb') as f:
        pickle.dump(sp_data[i], f)
    f.close()

## Constituents Close Prices

### S&P 500 Index

Eyeball

In [113]:
sp_data['sp500'].close.shape

(1258, 505)

In [114]:
sp_data['sp500'].close.head()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2014-05-08,37.6614,36.4381,121.4206,77.2831,43.3480,59.5766,20.87,34.8215,71.2567,59.08,...,26.3102,41.2198,84.8294,44.3080,26.9330,34.8308,49.6593,95.3575,27.3469,29.3889
2014-05-09,37.7227,36.4477,123.5028,76.9614,43.4394,60.6682,21.59,35.0552,71.1754,59.59,...,25.9390,41.3795,85.0965,44.7737,26.7522,34.7281,49.9143,95.9139,27.4130,29.4467
2014-05-12,38.4174,37.5241,124.1869,77.9193,43.5308,61.6766,21.92,35.3070,71.5821,60.70,...,25.7028,42.1741,85.3302,45.0550,27.1364,35.5124,49.7835,96.7773,27.9511,29.7071
2014-05-13,38.7034,37.4479,122.3625,78.0415,43.3895,61.8431,21.55,35.6576,72.0251,60.81,...,25.7872,41.2278,85.4387,45.1908,27.3171,35.4471,50.0059,97.1610,27.5924,29.6299
2014-05-14,38.0360,37.0002,122.0948,78.0560,43.9464,62.2594,21.55,35.9004,71.2748,60.88,...,26.0234,40.9064,85.3803,44.7834,27.0686,35.2603,49.6070,96.8253,26.8278,29.5431


In [115]:
sp_data['sp500'].close.tail()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2019-05-01,77.47,33.8124,163.83,210.52,78.89,73.39,277.07,78.74,181.31,283.35,...,55.90,116.64,78.67,50.31,33.26,83.29,101.94,121.22,48.58,101.79
2019-05-02,78.20,33.8523,166.78,209.15,78.47,77.06,264.77,78.78,179.39,279.64,...,55.92,118.90,77.29,51.50,32.95,79.54,101.74,123.21,49.48,103.15
2019-05-03,79.29,34.6899,163.27,211.75,78.71,79.14,271.75,78.69,176.98,285.58,...,56.58,119.02,77.47,55.05,32.95,82.29,102.72,124.31,50.05,103.75
2019-05-06,79.35,34.6500,161.99,208.48,79.26,78.31,268.95,79.07,176.26,283.66,...,56.51,118.81,77.13,54.73,32.65,80.08,102.41,125.55,49.68,103.33
2019-05-07,76.67,33.9100,160.66,202.86,77.95,77.46,261.98,76.91,173.94,277.07,...,56.58,117.81,76.72,54.83,32.67,79.17,101.47,123.39,48.71,101.37


In [116]:
sp_data['sp500'].close.describe()

Symbols,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,...,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,51.436064,41.345792,144.795082,133.694730,66.036601,84.724516,158.335584,48.359469,116.555211,138.371908,...,38.763042,57.830212,76.474149,53.008374,26.791466,50.707729,65.916667,111.362103,36.611229,58.447355
std,13.755806,6.214506,24.256595,38.308755,17.889930,9.869993,117.255513,12.193901,30.706247,68.803933,...,8.117917,21.417147,4.888988,7.895910,3.136379,16.178074,15.435964,10.042079,10.819180,19.704602
min,32.258600,24.539800,79.168700,76.961400,42.066600,59.576600,20.870000,33.935700,68.852300,59.080000,...,25.247700,32.609200,59.643400,34.178400,18.532600,28.968100,44.366900,89.236100,19.008100,29.196000
25%,39.216075,36.834425,130.475450,103.257075,52.496000,77.583825,75.590000,39.125525,90.534700,80.537500,...,31.167550,41.400900,73.431000,46.946725,24.215550,35.092925,52.717000,102.281950,26.912550,44.299275
50%,45.616500,40.848200,149.171500,118.630600,58.004100,84.243350,116.930000,42.802900,111.552950,105.400000,...,38.601350,51.019800,76.851000,53.931350,26.573400,47.791450,61.586100,112.401250,31.098600,50.322700
75%,65.149475,46.119900,161.834300,165.978000,82.894300,90.312650,236.005000,57.476725,147.724750,197.415000,...,45.139450,67.991325,79.636900,59.650550,29.472675,66.752175,79.165325,118.927750,46.604300,76.694475
max,81.940000,57.586600,199.159900,230.275400,116.445400,108.207500,449.750000,79.733700,182.670000,289.250000,...,57.360000,139.720000,87.124800,67.795300,35.000000,83.820000,104.390000,130.912800,57.511000,103.750000


## S&P MidCap 400 Index

Eyeball

In [117]:
sp_data['sp400'].close.shape

(1274, 400)

In [118]:
sp_data['sp400'].close.head()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2014-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-05-08,31.2321,32.1725,41.71,13.2125,31.85,NaN,9.5274,48.9195,52.6251,42.6417,...,22.2154,21.81,43.0260,15.4530,28.1817,24.2915,23.24,406.3601,53.29,74.51
2014-05-09,31.6652,31.9756,42.21,13.4225,32.07,NaN,9.8177,48.8031,52.6060,42.2125,...,22.0474,21.94,43.5655,15.3995,28.3556,23.8807,23.75,407.6382,54.22,74.03
2014-05-12,32.7774,31.9510,43.03,13.9825,32.60,NaN,9.9030,49.1525,52.5488,42.1031,...,22.1535,22.06,44.9818,16.2560,28.7706,24.7804,24.59,412.2492,56.60,74.23
2014-05-13,32.2164,31.6063,43.06,13.8000,32.02,NaN,10.0738,49.0277,52.5870,41.9076,...,22.0474,21.88,44.5868,16.4166,28.7152,24.8187,23.79,412.9473,55.53,73.41


In [119]:
sp_data['sp400'].close.tail()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
2019-05-01,54.63,47.11,32.37,34.85,33.47,23.02,23.82,103.06,70.06,80.40,...,38.61,19.89,108.17,82.63,44.48,15.27,67.13,656.90,39.64,205.12
2019-05-02,56.94,47.15,32.26,34.98,33.11,23.90,24.04,102.49,74.93,80.30,...,38.39,20.31,108.52,85.60,43.71,14.39,64.10,655.30,40.19,205.94
2019-05-03,58.75,47.31,33.42,35.42,33.62,24.89,24.12,103.51,74.82,82.05,...,38.09,22.96,112.34,85.80,44.63,16.88,64.45,662.82,40.98,206.46
2019-05-06,59.21,47.11,33.82,35.44,33.68,24.35,23.52,104.22,74.15,82.99,...,37.80,23.14,111.50,87.16,44.58,16.63,65.48,666.31,40.73,206.36
2019-05-07,58.28,46.25,32.60,34.40,33.11,21.97,22.71,102.42,73.11,82.20,...,37.88,22.54,109.12,84.93,43.92,16.41,62.51,652.75,39.91,200.82


In [120]:
sp_data['sp400'].close.describe()

Symbols,AAN,ACC,ACHC,ACIW,ACM,ADNT,AEO,AFG,AGCO,ALE,...,WTR,WW,WWD,WWE,WYND,X,XPO,Y,YELP,ZBRA
count,1258.000000,1258.000000,1258.000000,1258.00000,1258.000000,632.000000,1258.000000,1258.000000,1258.000000,1258.000000,...,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000
mean,34.931015,39.206532,49.006192,22.32030,32.395525,52.590945,15.613267,76.992750,55.521592,59.185235,...,29.486728,29.818931,62.629492,30.570165,36.770498,24.775982,53.522019,534.095435,40.559380,104.280874
std,9.453186,4.989326,13.752926,4.04358,3.178285,20.108995,4.051084,21.219234,9.356576,13.545504,...,4.837998,23.447140,14.966190,25.039742,7.051567,8.709335,25.865789,71.292458,13.389318,41.216138
min,20.118600,28.081400,24.750000,13.21250,23.150000,12.570000,8.872200,46.728500,40.309000,38.117400,...,20.741700,3.780000,38.963800,9.233700,25.207900,6.475600,19.560000,400.952700,15.230000,46.930000
25%,25.957675,34.676850,38.800000,19.39000,30.370000,40.785150,12.768500,58.717050,47.042025,45.528200,...,24.337000,12.060000,48.481650,15.568325,30.980500,19.104150,33.185000,471.816475,32.002500,74.200000
50%,34.368500,39.895300,46.660000,22.05000,32.415000,58.772600,14.581900,67.975550,53.301250,58.280900,...,29.693300,21.115000,61.785700,19.164850,34.604350,23.918500,45.865000,535.734250,39.565000,90.660000
75%,42.055425,43.660400,59.717500,24.04000,34.537500,66.653800,17.838250,97.214800,63.212425,73.280800,...,33.330300,44.240000,74.377500,34.221500,42.886600,32.300300,64.747500,598.713075,45.880000,123.157500
max,59.210000,48.853300,82.970000,35.52000,40.130000,84.046800,28.421700,114.972900,74.930000,83.830000,...,39.060000,103.090000,112.340000,99.250000,54.884400,45.690400,114.540000,666.310000,84.960000,235.440000


## S&P SmallCap 600 Index

Eyeball

In [121]:
sp_data['sp600'].close.shape

(1258, 601)

In [122]:
sp_data['sp600'].close.head()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
date,,,,,,,,,,,,,,,,,,,,,
2014-05-08,19.00,19.3403,30.3973,35.78,13.14,19.3536,61.36,23.9463,NaN,6.16,...,NaN,19.8117,77.73,9.1766,50.1827,24.8283,NaN,17.8549,23.4036,27.54
2014-05-09,17.57,19.6925,30.3797,35.88,13.26,19.7560,62.85,24.2267,NaN,6.48,...,NaN,19.9735,77.30,9.2092,51.2889,24.7151,NaN,18.3379,23.3449,28.28
2014-05-12,17.44,20.4546,30.1511,36.36,14.06,20.1009,64.66,24.8687,NaN,6.76,...,NaN,20.0868,79.30,9.2678,52.4801,25.1019,NaN,18.5836,24.3523,28.93
2014-05-13,17.26,19.8847,30.1248,37.40,13.64,19.9764,63.44,24.5703,NaN,6.56,...,NaN,19.8683,79.31,9.1571,52.0263,24.7246,NaN,18.5243,23.6579,28.52
2014-05-14,17.22,19.2827,29.9577,36.51,13.01,19.4878,62.94,23.9554,NaN,6.24,...,11.9624,19.8279,77.66,9.1115,50.9485,24.1208,NaN,18.1260,22.7092,27.72


In [123]:
sp_data['sp600'].close.tail()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
date,,,,,,,,,,,,,,,,,,,,,
2019-05-01,12.27,49.87,46.22,47.59,63.79,36.12,79.18,36.63,31.02,21.12,...,4.42,28.13,126.81,12.54,84.74,36.56,21.55,24.62,16.01,25.45
2019-05-02,12.09,49.52,46.12,45.91,64.26,36.75,79.07,37.42,30.29,21.27,...,4.39,27.89,129.49,12.50,84.36,37.04,22.31,24.88,16.14,25.79
2019-05-03,12.60,51.81,46.22,46.85,65.30,37.58,80.45,38.36,37.43,21.96,...,4.77,28.36,133.86,12.73,86.49,37.50,22.99,25.47,17.61,26.94
2019-05-06,12.69,49.35,46.18,44.95,65.84,37.39,80.58,38.41,37.62,22.08,...,4.86,28.36,132.62,12.78,87.51,36.98,23.00,25.08,17.02,26.53
2019-05-07,12.08,47.38,45.06,43.66,64.64,36.63,79.80,37.96,36.60,21.96,...,4.77,27.66,130.74,12.62,85.18,35.71,22.62,24.73,16.45,26.12


In [124]:
sp_data['sp600'].close.describe()

Symbols,AAOI,AAON,AAT,AAWW,AAXN,ABCB,ABG,ABM,ACA,ACLS,...,WPG,WRE,WRLD,WSR,WTS,WWW,XHR,XPER,ZEUS,ZUMZ
count,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,129.000000,1258.000000,...,1253.000000,1258.000000,1258.000000,1258.000000,1258.000000,1258.000000,1071.000000,1258.000000,1258.000000,1258.000000
mean,26.703816,29.202294,37.097485,49.382647,30.608104,36.171615,67.217234,32.512624,29.616171,15.849523,...,7.593659,26.063007,74.458859,10.773401,63.043193,26.629016,17.374759,26.417398,18.887357,22.918708
std,16.989437,7.527934,3.535975,10.208544,15.330705,10.737757,10.026713,5.655496,2.835425,6.635241,...,1.936801,3.243236,27.015761,1.389152,10.440913,5.742994,3.189206,6.745746,4.451113,6.831368
min,8.380000,16.343700,29.352300,31.400000,10.500000,18.903300,45.070000,22.347500,20.930200,6.160000,...,4.390000,19.787400,26.700000,7.392600,43.328400,14.741000,10.508700,12.051000,8.168700,11.450000
25%,15.067500,22.102250,35.161375,40.885000,21.942500,25.633650,59.167500,27.866050,28.614500,10.520000,...,6.081400,22.940600,50.137500,9.810950,54.086750,23.000950,14.879850,21.065900,16.500050,17.500000
50%,20.035000,29.215800,37.420750,49.150000,24.995000,34.582850,67.220000,31.081150,29.701800,13.650000,...,7.018500,26.387450,76.692500,10.819550,60.836300,26.391900,17.724600,26.908300,19.279500,22.075000
75%,33.927500,35.241675,38.993775,56.542500,33.760000,45.666850,73.450000,37.387475,31.059600,20.950000,...,8.849000,29.164850,97.420000,11.703750,71.644850,30.369425,19.503400,30.757175,22.269825,27.045000
max,99.610000,51.810000,46.770000,74.000000,74.890000,57.403000,95.540000,43.220900,37.620000,36.625000,...,12.678000,31.602900,133.860000,14.214700,87.510000,39.469500,24.438200,42.148900,30.787100,41.400000


## Summary

In this article, we collected historical data for each constituent of the S&P 500, S&P MidCap 400 and S&P SmallCap 600 indices using Python package pandas-datareader.